In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torch.multiprocessing as mp

import pdb

/home/alan/Desktop/envs/pao_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
is_cuda = False
dev = torch.device('cpu')  
if torch.cuda.is_available():
    is_cuda = True
    dev = torch.device('cuda')  

In [3]:
N = 1000

In [4]:
x1 = np.random.normal(3, 2, N)
x2 = np.random.normal(0, 5, N)

X = np.vstack((x1, x2)).T

# Noise
n1 = np.random.normal(0, 2, N)
n2 = np.random.normal(0, 6, N)
n3 = np.random.normal(0, 10, N)

y1 = x1**2 + 5*x1*x2 + 3*np.abs(x2) + n1

In [5]:
X_train = torch.tensor(X[:800], dtype = torch.float32, requires_grad = True, device = dev)
y1_train = torch.tensor(y1[:800],  dtype = torch.float32, requires_grad = True, device = dev)

X_test = torch.tensor(X[800:],  dtype = torch.float32, requires_grad = True, device = dev)
y1_test = torch.tensor(y1[800:],  dtype = torch.float32, requires_grad = True, device = dev)

In [6]:
X_train.type()

'torch.FloatTensor'

In [7]:
class VariationalLayer(nn.Module):
    def __init__(self, 
                 input_size, output_size,
                 prior_mu, prior_rho,
                 n_samples
                ):
        super().__init__()
        
        # Bias weight
        input_size = input_size + 1
        
        # Defining Prior distribution (Gaussian)
        self.prior_mu = torch.tensor(prior_mu)
        self.prior_rho = torch.tensor(prior_rho)
        
        # Defining Variational class (Gaussian class)
        self.theta_mu = nn.Parameter(
            torch.Tensor(n_samples, input_size, output_size).uniform_(-0.2, 0.2)).float()
        self.theta_rho = nn.Parameter(
            torch.Tensor(n_samples, input_size, output_size).uniform_(-2,-1)).float()
        
        # Defining some constants
        self.logsqrttwopi = torch.log(
            torch.sqrt(2*torch.tensor(torch.pi)))
        self.K = torch.tensor(1)
        
        # Defining number of samples for forward
        self.n_samples = n_samples
    
    def rho_to_sigma(self, theta_rho):
        return torch.log(1 + torch.exp(theta_rho))

    def sample_weight(self, theta_mu, theta_rho):
        w = (theta_mu 
        + self.rho_to_sigma(theta_rho)*torch.randn(theta_mu.shape))
        return w

    def log_prob_gaussian(self, x, mu, rho):
            return (
                - self.logsqrttwopi
                - self.rho_to_sigma(rho)
                - ((x - mu)**2)/(2*self.rho_to_sigma(rho)**2)
            ).sum()
    
    def prior(self, w):
        return self.log_prob_gaussian(
            w, self.prior_mu, self.prior_rho)
        
    def variational(self, w, theta_mu, theta_rho):
        return self.log_prob_gaussian(
            w, theta_mu, theta_rho) 
    
    def kl_divergence_layer(self):
        theta_mu = self.theta_mu
        theta_rho = self.theta_rho
        w = self.sample_weight(theta_mu, theta_rho)
        Q = self.variational(w, theta_mu, theta_rho)
        P = self.prior(w)
        KL = Q - P 
        return KL
    
    def forward(self, x_layer):
        theta_mu = self.theta_mu
        theta_rho = self.theta_rho
        w = self.sample_weight(theta_mu, theta_rho)    
        x_next_layer = torch.bmm(x_layer, w[:, :-1, :]) + w[:,-1,:].unsqueeze(1)
        return x_next_layer

In [140]:
class VariationalNet(nn.Module):
    # Initialize the layers
    def __init__(self, n_samples, input_size, output_size):
        super().__init__()
        self.n_samples = n_samples
        self.act1 = nn.ReLU()
        self.linear1 = VariationalLayer(input_size, 5, 0, 0.54, n_samples)
        #self.bn = nn.BatchNorm1d(5)
        self.linear2 = VariationalLayer(5, output_size, 0, 0.54, n_samples)
    
    # Perform the computation
    def forward(self, x):
        x = torch.unsqueeze(x, 0)
        x = x.expand((self.n_samples, x.shape[1], x.shape[2]))
        x = self.linear1(x)
        #x = self.bn(x)
        x = self.act1(x)
        x = self.linear2(x)
        #x = self.act1(x)
        return x
    
    def kl_divergence_NN(self):
        kl = self.linear1.kl_divergence_layer() + self.linear2.kl_divergence_layer()
        return kl

In [223]:
n_samples = 3
input_size = X_train.shape[1]
output_size = 1

h = VariationalNet(n_samples, input_size, output_size).to(dev)
opt_h = torch.optim.Adam(h.parameters(), lr=0.01)

mse_loss = nn.MSELoss(reduction='mean')

#X_train = torch.unsqueeze(X_train, 2)
index = np.random.randint(0, 800, 5)
x_h = X_train[index]
y_h = y1_train[index]
h.forward(x_h)

In [386]:
for i in range(0, 10):
    index = np.random.randint(0, 800, 50)
    x_h = X_train[index]
    y_h = y1_train[index]
    
    opt_h.zero_grad()
    y_preds = h(x_h)
    y_preds_mean = y_preds.mean(axis=0).squeeze()
    elbo_loss = mse_loss(y_preds_mean, y_h) + h.kl_divergence_NN()
    elbo_loss.backward()
    opt_h.step()
    
    print(mse_loss(y_preds_mean, y_h), h.kl_divergence_NN())

tensor(324.0914, grad_fn=<MseLossBackward0>) tensor(340.0129, grad_fn=<AddBackward0>)
tensor(346.3026, grad_fn=<MseLossBackward0>) tensor(343.7459, grad_fn=<AddBackward0>)
tensor(1903.8341, grad_fn=<MseLossBackward0>) tensor(340.8668, grad_fn=<AddBackward0>)
tensor(639.5687, grad_fn=<MseLossBackward0>) tensor(335.0477, grad_fn=<AddBackward0>)
tensor(445.5335, grad_fn=<MseLossBackward0>) tensor(345.3464, grad_fn=<AddBackward0>)
tensor(803.3212, grad_fn=<MseLossBackward0>) tensor(338.1334, grad_fn=<AddBackward0>)
tensor(580.7350, grad_fn=<MseLossBackward0>) tensor(329.2625, grad_fn=<AddBackward0>)
tensor(561.3204, grad_fn=<MseLossBackward0>) tensor(334.4700, grad_fn=<AddBackward0>)
tensor(635.4660, grad_fn=<MseLossBackward0>) tensor(346.8938, grad_fn=<AddBackward0>)
tensor(1052.6262, grad_fn=<MseLossBackward0>) tensor(350.6403, grad_fn=<AddBackward0>)


In [221]:
mse_loss(y_preds_mean, y_h)

tensor(790.9849, grad_fn=<MseLossBackward0>)

In [222]:
h.kl_divergence_NN()

tensor(202.5808, grad_fn=<AddBackward0>)

In [219]:
y_preds_mean

tensor([ 102.2150,  145.6053,   20.7263,   -1.6152,  -31.5738,   10.1983,
          24.0533,   54.6786,  -23.9765,  -69.9426,  -81.3019,  -63.6686,
          92.4034,  131.3438,   99.1495, -111.5350,  -64.6963,  -50.1505,
         197.3893,   64.2947,   50.1944, -111.5350, -140.7539,   68.0634,
         -47.6772,  -27.9659,  190.0887,  -88.2236, -124.4992,    3.7649,
         -46.4705,   40.3146,  160.9212,  -50.7410,  -12.7711,  -72.0561,
         -42.7727,  -75.9347,  108.1896,  160.0007,   27.8732,  228.6359,
         151.4664,  126.0074,   94.7874,  -23.6355,   72.0101,   -5.6030,
         115.2242,   -5.5860], grad_fn=<SqueezeBackward0>)

In [220]:
y_h

tensor([  67.7424,  133.4991,   14.8095,    6.8667,  -10.0592,    1.8639,
          17.4458,   34.9345,  -17.4515,  -60.1327,  -74.0747,  -49.9519,
          70.7016,  130.5905,   71.5508, -148.9616,  -49.9473,  -18.4058,
         237.8830,   51.3122,   14.6421, -148.9616, -246.1836,   55.9984,
         -13.2758,  -34.7388,  145.3001, -127.8421, -188.8626,    5.0664,
         -54.7836,   23.9301,  148.0581,  -41.7343,    2.5848,  -38.4266,
         -18.1399,  -72.8139,  109.4105,  179.5030,   16.9142,  288.7208,
         118.8489,  142.0373,   93.5721,  -16.7625,   66.8598,    3.4595,
         110.7035,    0.8648], grad_fn=<IndexBackward0>)

In [114]:
for name, param in h.named_parameters():
    if param.requires_grad:
        print (name, param.data)

linear1.theta_mu tensor([[[ 0.8771,  0.8008,  0.5494,  0.6440,  0.4596],
         [-0.4487,  0.7498,  0.5977,  0.2930, -0.1289],
         [ 0.8443,  0.6756,  0.3603,  0.5161,  0.5984]],

        [[ 0.7617,  0.4910,  0.6485,  0.7612,  0.8183],
         [ 0.2891, -0.4360, -0.4808,  0.7688, -0.4957],
         [ 0.8583,  0.6387,  0.6754,  0.6904,  0.7762]],

        [[ 0.6100,  0.7565,  0.5851,  0.4265,  0.3386],
         [ 0.1600,  0.5227, -0.3907,  0.5654,  0.0436],
         [ 0.5872,  0.8885,  0.4619,  0.6182,  0.5223]]])
linear1.theta_rho tensor([[[-1.5920, -1.2908, -1.5692, -0.9557, -0.8429],
         [-0.8280, -1.4659, -1.1269, -1.2181, -1.1346],
         [-1.5434, -1.2956, -1.3341, -1.1154, -1.1518]],

        [[-1.3310, -1.8959, -1.3955, -1.7491, -1.6686],
         [-0.7042, -1.4796, -0.7514, -1.0054, -1.0801],
         [-1.7647, -1.1682, -1.3814, -1.2318, -1.2843]],

        [[-1.1970, -1.0642, -1.5848, -1.3141, -1.0779],
         [-1.5442, -1.5914, -1.0839, -1.3320, -0.5853],
   

In [13]:
elbo_loss

tensor(989.9192, grad_fn=<AddBackward0>)

In [12]:
input_size = 2
output_size = 3
vi_obj = VariationalLayer(input_size, output_size, 0, 0.54)

In [13]:
#X_train = torch.unsqueeze(X_train, 2)
index = np.random.randint(0, 800, 4)
x_h = X_train[index]
y_h = y1_train[index]

In [15]:
vi_obj.forward(x_h, 10).shape

torch.Size([5, 3, 10])

In [33]:
elbo_loss(xh, yh, h, w)

NameError: name 'elbo_loss' is not defined

In [192]:
#X_train = torch.unsqueeze(X_train, 2)

In [162]:
w = vi_obj.sample_weight(weight_mu, weight_rho)
Q = vi_obj.variational(w, weight_mu, weight_rho)
P = vi_obj.variational(w)

index = np.random.randint(0, 800, 5)
x_h = X_train[index]
y_h = y1_train[index]
l = mse_loss(h(x_h), y_h)

TypeError: variational() missing 2 required positional arguments: 'theta_mu' and 'theta_rho'

In [193]:
index = np.random.randint(0, 800, 5)
x_h = X_train[index]
y_h = y1_train[index]
l = mse_loss(h(x_h), y_h)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x1 and 2x6)

In [124]:
x_h.shape

torch.Size([200, 2])

In [130]:
h.forward_parallel(x_h)

RuntimeError: Cowardly refusing to serialize non-leaf tensor which requires_grad, since autograd does not support crossing process boundaries.  If you just want to transfer the data, call detach() on the tensor before serializing (e.g., putting it on the queue).

In [58]:
X_train

array([[ 1.41972735,  1.2029036 ],
       [ 3.64036384, -1.80001503],
       [ 1.66258872, -9.45797395],
       ...,
       [ 3.35859904,  2.77637386],
       [ 2.11101038,  5.22272453],
       [ 2.69671436,  2.78818971]])

In [ ]:
P = vi_obj.prior(vi_obj.sample_weight(weight_mu, weight_rho))

In [112]:
index = np.random.randint(0, 800, 200)
x_h = X_train[index]
y_h = y1_train[index]

In [113]:
x_h

tensor([[ 4.2213e+00, -6.2528e+00],
        [ 2.7539e+00,  2.2754e+00],
        [ 4.8483e+00, -3.6463e+00],
        [ 4.6008e+00, -2.2948e+00],
        [ 6.7030e+00, -2.3596e+00],
        [ 1.8571e+00,  6.8027e-01],
        [ 2.9115e+00, -5.4428e+00],
        [ 2.3574e+00, -4.6242e+00],
        [ 4.1867e+00, -5.0198e+00],
        [ 5.5665e+00,  1.7493e+00],
        [ 4.1867e+00, -5.0198e+00],
        [ 1.6467e+00, -1.4431e+00],
        [ 1.7152e+00, -2.6472e+00],
        [ 4.8483e+00, -3.6463e+00],
        [ 3.3220e+00,  3.2644e+00],
        [ 1.7996e+00, -6.0863e+00],
        [ 9.8054e-01,  3.7539e+00],
        [ 5.2619e+00, -2.2712e+00],
        [ 5.8081e+00,  4.0490e+00],
        [ 5.0169e+00,  3.7023e+00],
        [ 7.5814e-01,  3.0518e-01],
        [ 4.2604e+00,  5.6789e+00],
        [ 7.5471e+00, -1.5020e+00],
        [ 6.1520e+00,  4.9272e+00],
        [ 4.3850e+00,  2.8569e+00],
        [ 4.5348e+00,  3.5263e+00],
        [ 3.4467e+00, -4.1564e+00],
        [-1.3712e+00,  3.650

In [114]:
np.random.randint(0,N)

463

In [115]:
mse_loss(h(x_h), y_h)

tensor(8096.7533, dtype=torch.float64, grad_fn=<MseLossBackward0>)

In [174]:
w = sample_weight(weight_mu, weight_rho)

In [175]:
w

tensor([[-0.0181, -0.4754]], grad_fn=<AddBackward0>)

In [176]:
prior(w)

tensor(-3.9496, grad_fn=<SumBackward0>)

In [177]:
variational_posterior(w, weight_mu, weight_rho)

tensor(-4.1615, grad_fn=<SumBackward0>)